In [1]:
from itertools import product
import numpy as np
import pandas as pd

# Подключаем старые пакеты для моделирования протокола RFID
from pysim.models.monte_carlo.protocol import (TagFrame, ReaderFrame, TagEncoding,
    Sel, DR, Session, Bank, InventoryFlag as Flag,
    Query, QueryRep, Ack, ReqRn, Read,
    Rn16Reply, AckReply, ReqRnReply, ReadReply, max_t1, max_t2)

from pysim.models.monte_carlo.cli import cli_run

In [2]:
# Параметры модели
BER = 0.001
CHUNKS_NUMBER = 2
WORDS_NUMBER = 61

# Параметры протокола RFID
DRS = [DR.DR_8]
M = [TagEncoding.FM0]
TARI_US = [6.25]
TRExt = (False,)

In [3]:
words_number_in_one_chunk = WORDS_NUMBER
last_chunk_len = 0
number_chunks_to_send = CHUNKS_NUMBER
if CHUNKS_NUMBER > 1:
    words_number_in_one_chunk = WORDS_NUMBER // CHUNKS_NUMBER
    last_chunk_len = WORDS_NUMBER // CHUNKS_NUMBER + WORDS_NUMBER % CHUNKS_NUMBER
        
print(f'Слов в одном чанке: {words_number_in_one_chunk}')
print(f'Слов в последнем чанке: {last_chunk_len}')

Слов в одном чанке: 30
Слов в последнем чанке: 31


In [4]:
def random_hex_string(bs: int) -> str:
    '''
    Для генерации случайного EPC и TID
    '''
    return "".join([f"{np.random.randint(0, 256):02X}" for _ in range(bs)])

In [5]:
Q = 4             # по-умолчанию, будем исходить из этого значения параметра Q
RTCAL_MUL = 2.75  # во сколько раз RTcal больше Tari
TRCAL_MUL = 1.95  # во сколько раз TRcal больше RTcal
EPC_SIZE = 12     # длина EPCID в байтах
TID_SIZE = 8      # длина TID в байтах

EPC = random_hex_string(EPC_SIZE)
TID = random_hex_string(TID_SIZE)

print("Будут использованы следующие константы:")
print(f"- EPC: {EPC} ({EPC_SIZE} bytes)")
print(f"- TID: {TID} ({TID_SIZE} bytes)")

Будут использованы следующие константы:
- EPC: 650E95BE7A39BE8033C36AAA (12 bytes)
- TID: 1284731B53F20C6F (8 bytes)


In [6]:
def build_messages_df() -> pd.DataFrame:
    """
    Построить DataFrame для всевозможных настроек канала и рассчитанными 
    длительностями команд и ответов.
    
    В датафрейме используются следующие единицы измерений:
    
    - для длительностей: микросекунды (мкс)
    - для частот: килогерцы (КГц)
    - для скоростей: килобиты в секунду (кбит/с)
    
    Returns:
        df (DataFrame)
    """
    params = []
    for tari_us, m, dr, trext in product(TARI_US, M, DRS, TRExt):
        tari = tari_us * 1e-6
        rtcal = RTCAL_MUL * tari
        trcal = TRCAL_MUL * rtcal
        blf = dr.ratio / trcal
        bitrate = blf / m.value
        
        # Строим команды
        # --------------
        preamble = ReaderFrame.Preamble(tari, rtcal, trcal)
        sync = ReaderFrame.Sync(tari, rtcal)
        
        query = Query(dr=dr, m=m, trext=trext, sel=Sel.SL_ALL, 
                           session=Session.S0, target=Flag.A, 
                           q=Q, crc5=0x15)
        query_rep = QueryRep(session=Session.S0)
        ack = Ack(0x5555)
        req_rn = ReqRn(0x5555, 0x5555)
        read = Read(Bank.TID, 0, 4, rn=0x5555, crc16=0x5555)
        
        # Строим ответы
        # -------------
        rn16 = Rn16Reply(0x5555)
        epc_pc = AckReply(EPC)
        handle = ReqRnReply(0)
        data = []
        if CHUNKS_NUMBER > 1:
            for i in range(CHUNKS_NUMBER-1):
                data.append(ReadReply([0xABAB] * words_number_in_one_chunk, 0, 0))
            data.append(ReadReply([0xABAB] * last_chunk_len, 0, 0))
        else:
            data = [ReadReply([0xABAB] * WORDS_NUMBER, 0, 0)]
        data_durations = []
        data_lens = []
        for data_answer in data:
            data_durations.append(TagFrame(m, trext, blf, data_answer).duration)
            data_lens.append(TagFrame(m, trext, blf, data_answer).bitlen)

        params.append({
            'DR': dr,
            'M': m,
            'Tari': tari_us,     
            'TRext': trext,
            'RTcal': rtcal,
            'TRcal': trcal,
            'BLF': blf,
            'Query': ReaderFrame(preamble, query).duration,
            'QueryRep': ReaderFrame(sync, query_rep).duration,
            'Ack': ReaderFrame(sync, ack).duration,
            'Req_RN': ReaderFrame(sync, req_rn).duration,
            'Read': ReaderFrame(sync, read).duration,
            'Query_len': 22,
            'RN16': TagFrame(m, trext, blf, rn16).duration,
            'EPC+PC+CRC': TagFrame(m, trext, blf, epc_pc).duration,
            'Handle': TagFrame(m, trext, blf, handle).duration,
            'Data': data_durations,
            'RN16_len': TagFrame(m, trext, blf, rn16).bitlen,
            'EPC+PC+CRC_len': TagFrame(m, trext, blf, epc_pc).bitlen,
            'Handle_len': TagFrame(m, trext, blf, handle).bitlen,
            'Data_len': data_lens,
        })
    return params

params = build_messages_df()
DF = pd.DataFrame(params)
params

[{'DR': <DR.DR_8: 0>,
  'M': <TagEncoding.FM0: 1>,
  'Tari': 6.25,
  'TRext': False,
  'RTcal': 1.7187499999999998e-05,
  'TRcal': 3.3515624999999996e-05,
  'BLF': 238694.6386946387,
  'Query': 0.00023039062499999996,
  'QueryRep': 6.09375e-05,
  'Ack': 0.00019062499999999998,
  'Req_RN': 0.00037499999999999995,
  'Read': 0.0004968749999999999,
  'Query_len': 22,
  'RN16': 9.6357421875e-05,
  'EPC+PC+CRC': 0.0005655761718749999,
  'Handle': 0.000163388671875,
  'Data': [0.002178515625, 0.0022455468749999997],
  'RN16_len': 23,
  'EPC+PC+CRC_len': 135,
  'Handle_len': 39,
  'Data_len': [520, 536]}]

Вероятность успешного перехода из одного состояния в другое определяется
исключительно ответами метки (так как во всём проекте сделано допущение, что
если метка оказалась в зоне чтения, то она гарантировано включается, корректно
получает команду считывателя и принимает участие в текущем раунде). Вероятность
перехода в следующее состояние равна вероятности успешного получения считывателем
ответа метки.
Вероятность определяется выражением:
$$ P_{msg} = (1 - BER)^{|msg|} $$
где msg - передаваемое сообщение, BER - вероятность битовой ошибки, |msg| - длина сообщения в битах

In [7]:
reply_lens = [
    params[0]['RN16_len'],
    params[0]['EPC+PC+CRC_len'],
    params[0]['Handle_len'],
]
for i in range(CHUNKS_NUMBER):
    reply_lens.append(params[0]['Data_len'][i])
reply_lens

[23, 135, 39, 520, 536]

In [8]:
probabilities = []
for i in range(len(reply_lens)):
    probabilities.append((1 - BER) ** reply_lens[i])
probabilities

[0.9772512378214517,
 0.8736568985103138,
 0.9617319426784903,
 0.5943658896200157,
 0.5849270275271061]

In [9]:
T1 = max_t1(params[0]['RTcal'], params[0]['BLF'])
T2 = max_t2(params[0]['BLF'])
T1_2 = T1 + T2
delta = 1e-5

In [10]:
times = []
T_arbitrate = T1_2 + params[0]['Query'] + params[0]['RN16'] + 2 * delta
times.append(T_arbitrate)
T_reply = T1_2 + params[0]['Ack'] + params[0]['EPC+PC+CRC'] + 2 * delta
times.append(T_reply)
T_acknowledged = T1_2 + params[0]['Req_RN'] + params[0]['Handle'] + 2 * delta
times.append(T_acknowledged)

for i in range(CHUNKS_NUMBER):
    t_chunk = T1_2 + params[0]['Read'] + params[0]['Data'][i] + 2 * delta
    times.append(t_chunk)
times

[0.00047862109374999993,
 0.0009080742187499999,
 0.0006902617187499999,
 0.002827263671875,
 0.0028942949218749995]

In [11]:
res = cli_run()

KeyError: 'scenario'